In [15]:
#!pip install arxiv
#!pip install wikipedia 
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
# Tool-1 : WIKIPEDIA TOOL
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)

In [ ]:
# Tool-2 : Website Data loader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector_db = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector_db.as_retriever()

# Create a Tool using the retriever
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith_search", 
                                      "Search for langsmith using this tool. For any questions on Langsmith, you must use this tool")

In [39]:
# Tool-3 : For Arxiv website 
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200))

In [40]:
# Combine these tools
tools = [wiki_tool, arxiv_tool, retriever_tool]

In [41]:
# Create prompt. We can use ChatPromptTemplate as well as use Langchain Hub where predefined generic prompts are available.
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

In [42]:
# Define the LLM model
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


In [43]:
# Agent
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(model,tools, prompt)

# Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [44]:
# Execute
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Langsmith'}`


No good Wikipedia Search Result was foundI couldn't find any information about Langsmith on Wikipedia. Would you like me to search for Langsmith using a different tool?

> Finished chain.


{'input': 'Tell me about Langsmith',
 'output': "I couldn't find any information about Langsmith on Wikipedia. Would you like me to search for Langsmith using a different tool?"}

In [45]:
agent_executor.invoke({"input" : "tell me about the latest search paper on computer vision"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'computer vision'}`


Published: 2019-05-20
Title: Implications of Computer Vision Driven Assistive Technologies Towards Individuals with Visual Impairment
Authors: Linda Wang, Alexander Wong
Summary: Computer vision basedThe latest search paper on computer vision is titled "Implications of Computer Vision Driven Assistive Technologies Towards Individuals with Visual Impairment" by Linda Wang and Alexander Wong. The paper discusses the implications of computer vision-driven assistive technologies for individuals with visual impairment. It was published on May 20, 2019.

> Finished chain.


{'input': 'tell me about the latest search paper on computer vision',
 'output': 'The latest search paper on computer vision is titled "Implications of Computer Vision Driven Assistive Technologies Towards Individuals with Visual Impairment" by Linda Wang and Alexander Wong. The paper discusses the implications of computer vision-driven assistive technologies for individuals with visual impairment. It was published on May 20, 2019.'}

In [46]:
agent_executor.invoke({"input" : "tell me about the power of spider man"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'power of spider man'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Get started by adding tracing to your application.
Create dashboards to view key metrics like RPS, error rates and costs.

Evals​
The quality and development speed of AI applications depends on high-quality evaluation datasets and metrics to test and optimize your applications on. The LangSmith SDK and UI make building and running high-quality evaluations easy.

Get started by creating your first evaluation.
Quickly assess the performance of your application using our off-the-shelf evaluators as a starting point.
Analyze results of evaluations in the LangSmith UI and compare results over time.
Easily collect human feedback on your data to improve your application.

Prompt Engineering​
While traditional software applications are built by writing code, AI applications involve writing prompts to instruct the LLM on what to do. LangSmi

{'input': 'tell me about the power of spider man',
 'output': "I couldn't find information specifically about the power of Spider-Man. Would you like me to search for information on Spider-Man's abilities and powers instead?"}